## Download the model that you want to fine-tune from the Hugging Face model hub

In [1]:
from huggingface_hub import hf_hub_download
import os 
import shutil
from mescnn.classification.inference.download import download_classifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from PIL import Image
from tempfile import TemporaryDirectory

# Replace this with the path to your local directory
fine_tuning_model_dir = "/home/ubuntu/M1/Data/Fine_tuning/models/"
local_models_dir = "./mescnn/classification/logs/cnn/holdout"

# Create the directory if it does not exist
if not os.path.exists(fine_tuning_model_dir):
    os.makedirs(fine_tuning_model_dir)

# Automate for the 4 models (C, S, M, E) 
model_types = ["MESCnn", "Fine_tuned_MESCnn"]
for model_type in model_types:

    # Lesion types
    lesions = ["C", "S", "M", "E"]

    for lesion in lesions:
        # Download the model
        if model_type == "MESCnn":
            if (lesion == "C"):
                model_id = f"mobilenetv2_{lesion}_V3.pth" # For C lesion
            elif (lesion == "S"):
                model_id = f"densenet161_{lesion}_V3.pth" # For S lesion
            else:
                model_id = f"efficientnetv2-m_{lesion}_V3.pth" # For M & E lesion
        else:
            if (lesion == "C"):
                model_id = f"mobilenetv2_{lesion}_V3_fine_tuned.pth"
            elif (lesion == "S"):
                model_id = f"densenet161_{lesion}_V3_fine_tuned.pth"
            else:
                model_id = f"efficientnetv2-m_{lesion}_V3_fine_tuned.pth"


        model_name = model_id.split("/")[-1]
        splited_model_name = model_name.split("_")
        print(splited_model_name)
        local_model_path = os.path.join(local_models_dir, model_name)


        if not os.path.exists(local_model_path):
            print("Downloading model to {}...".format(local_model_path))
            downloaded_model = download_classifier(splited_model_name[0], splited_model_name[1], splited_model_name[2].split(".")[0])
            # Copy the model from the download directory to the fine_tune directory
            shutil.copy(downloaded_model, os.path.join(fine_tuning_model_dir, model_name))
            print("Download complete.")
        else:
            print("Model {} already exists in local directory. Skipping download.".format(model_id))
            shutil.copy(local_model_path, os.path.join(fine_tuning_model_dir, model_name))

        # Load the model
        net = torch.load(local_model_path)

        cudnn.benchmark = True
        plt.ion()   # interactive mode

        learning_rate = 10**(-5)
        momentum = 0.8
        batch_size = 4
        epochs = 80
        augmentation = ['HFlip', 'VFlip', 'BtnsCtst']
        frozen = False

        data_transforms = {
            'train': transforms.Compose([
                transforms.ToTensor(),
                transforms.RandomHorizontalFlip(p=0.25),
                transforms.RandomVerticalFlip(p=0.25),
                transforms.RandomApply([transforms.ColorJitter(brightness=0.3, contrast=0.3)], p=0.25),
            ]),
            'val': transforms.Compose([
                transforms.ToTensor(),
                transforms.RandomHorizontalFlip(p=0.25),
                transforms.RandomVerticalFlip(p=0.25),
                transforms.RandomApply([transforms.ColorJitter(brightness=0.3, contrast=0.3)], p=0.25),
            ]),
        }

        data_dir = "/home/ubuntu/M1/Data/Fine_tuning/Classification/" + lesion 
        image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
        dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val']}
        dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

        # Print the data, and each class, as well as the number associated with each class
        print(image_datasets)
        print(image_datasets['train'].classes)
        print(image_datasets['train'].class_to_idx)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Do the training multiple times to make the average of the accuracy
        list_global_accuracy = []
        for i in range(10):
            # Test on all the data to get the accuracy before fine-tuning
            for x in ['train', 'val']:
                correct = 0
                total = 0
                with torch.no_grad():
                    for data in dataloaders[x]:
                        images, labels = data
                        images, labels = images.to(device), labels.to(device)
                        outputs = net(images)
                        _, predicted = torch.max(outputs, 1)
                        # print(labels, outputs, predicted)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                print('Accuracy of the network on the {} {} images: {}%'.format(x, total, 100 * correct / total))
                if x == 'train':
                    nb_train_images = total
                    accuracy_train = correct / total
                else:
                    nb_val_images = total
                    accuracy_val = correct / total
        
            global_accuracy = (nb_train_images * accuracy_train + nb_val_images * accuracy_val) / (nb_train_images + nb_val_images)
            print('Global accuracy of the network on the {} images: {}%'.format(nb_train_images + nb_val_images, global_accuracy*100))
            # stock the global accuracy of the model
            list_global_accuracy.append(global_accuracy)
        average_accuracy = sum(list_global_accuracy) / len(list_global_accuracy)
        # Stock the results for the 4 models in a csv file
        with open("/Data/Fine_tuning/results.csv", "a") as f:
            f.write(f"Project,Lesion,Model,Architecture,Version,Global_Accuracy\n")
            f.write(f"{model_type},{lesion},{splited_model_name[0]},{splited_model_name[1]},{splited_model_name[2].split('.')[0]},{average_accuracy}\n")



/home/ubuntu/anaconda3/envs/FionnAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['mobilenetv2', 'C', 'V3.pth']
Model mobilenetv2_C_V3.pth already exists in local directory. Skipping download.
MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
